In [0]:
"""
https://delta.io/blog/2022-10-03-rollback-delta-lake-restore/

Delta Lake restore use cases
Having a versioned data lake is excellent for time travel, but it’s also invaluable for undoing mistakes. Delta Lake’s restore functionality provides great flexibility. You can use it to easily roll back unwanted operations, preserving a full change history.

Perhaps your data ingestion process broke, and you loaded the same data twice – you can undo this operation with a single command. You might also decide to roll back your Delta Lake table to a previous version because you executed a command with unintended consequences. Or perhaps a data vendor sent you some information mistakenly and they’d like you to delete it from your data lake – you can roll back to a prior version and then vacuum the table to remove the data permanently, as you'll see later in this post. In short, the Delta Lake restore functionality is yet another example of how Delta Lake makes your life easier as a developer.
"""

In [0]:
df = spark.range(0,3)
df.show()
df.write.format("delta").save("/FileStore/tables/delta-example/restore-example")

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [0]:
df2 = spark.range(4,6)
df2.show()
df2.write.format("delta").mode("overwrite").save("/FileStore/tables/delta-example/restore-example")

+---+
| id|
+---+
|  4|
|  5|
+---+



In [0]:
df3 = spark.range(7, 10)
df3.show()
df3.write.format("delta").mode("overwrite").save("/FileStore/tables/delta-example/restore-example")

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [0]:
spark.read.format("delta").load("/FileStore/tables/delta-example/restore-example").show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



# Delta Lake restore under the hood



In [0]:
# Restore our example Delta Lake table to version 1

from delta.tables import *
delta_table = DeltaTable.forPath(spark, "/FileStore/tables/delta-example/restore-example")
delta_table.restoreToVersion(1)

Out[9]: DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [0]:
spark.read.format("delta").load("/FileStore/tables/delta-example/restore-example").show()

"""
Since delta table was restored to 1st version(id with values 4,5) if we read delta table by default it will show 4, 5 as it has become the latest data now and latest data is retrieved by default.
"

+---+
| id|
+---+
|  4|
|  5|
+---+



In [0]:
spark.read.format("delta").option("versionAsOf", 3).load("/FileStore/tables/delta-example/restore-example").show()
"""
Note that restoreToVersion created another version of the data but version=1 data is still intact.
This means you can still time travel to versions 0, 1, or 2 of the Delta Lake table, even after running the restoreToVersion command.
Using the restore command resets the table’s content to an earlier version, but doesn’t remove any data. It simply updates the transaction log to indicate that certain files should not be read. 
"""

+---+
| id|
+---+
|  4|
|  5|
+---+



In [0]:
spark.read.format("delta").option("versionAsOf", 2).load("/FileStore/tables/delta-example/restore-example").show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [0]:
spark.read.format("delta").option("versionAsOf", 1).load("/FileStore/tables/delta-example/restore-example").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [0]:
spark.read.format("delta").option("versionAsOf", 0).load("/FileStore/tables/delta-example/restore-example").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+

